In [ ]:
# 回测：外汇K线数据

[NautilusTrader](https://nautilustrader.io/docs/) 高性能算法交易平台和事件驱动回测器教程。

[在GitHub上查看源码](https://github.com/nautechsystems/nautilus_trader/blob/develop/docs/tutorials/backtest_fx_bars.ipynb)。

:::info
我们目前正在完善此教程。
:::


In [ ]:
## 概述

本教程介绍如何使用外汇K线数据设置一个`BacktestEngine`（低级API）进行单次'一次性'回测运行。


In [ ]:
## 先决条件

- 已安装 Python 3.11+
- 已安装 [JupyterLab](https://jupyter.org/) 或类似工具 (`pip install -U jupyterlab`)
- 已安装 [NautilusTrader](https://pypi.org/project/nautilus_trader/) 最新版本 (`pip install -U nautilus_trader`)


In [ ]:
## 导入

我们先导入本教程所需的所有模块。


In [ ]:
from decimal import Decimal

from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.backtest.models import FillModel
from nautilus_trader.backtest.modules import FXRolloverInterestConfig
from nautilus_trader.backtest.modules import FXRolloverInterestModule
from nautilus_trader.config import LoggingConfig
from nautilus_trader.config import RiskEngineConfig
from nautilus_trader.examples.strategies.ema_cross import EMACross
from nautilus_trader.examples.strategies.ema_cross import EMACrossConfig
from nautilus_trader.model import BarType
from nautilus_trader.model import Money
from nautilus_trader.model import Venue
from nautilus_trader.model.currencies import JPY
from nautilus_trader.model.currencies import USD
from nautilus_trader.model.enums import AccountType
from nautilus_trader.model.enums import OmsType
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import TestDataProvider
from nautilus_trader.test_kit.providers import TestInstrumentProvider


In [ ]:
## 设置回测引擎


In [ ]:
# 初始化回测配置
config = BacktestEngineConfig(
    trader_id="BACKTESTER-001",
    logging=LoggingConfig(log_level="ERROR"),
    risk_engine=RiskEngineConfig(
        bypass=True,  # 回测中绕过交易前风险检查的示例
    ),
)

# 构建回测引擎
engine = BacktestEngine(config=config)


In [ ]:
## 添加仿真模块

我们可以选择性地插入一个模块来仿真展期利息。数据来自预打包的测试数据。


In [ ]:
provider = TestDataProvider()
interest_rate_data = provider.read_csv("short-term-interest.csv")
config = FXRolloverInterestConfig(interest_rate_data)
fx_rollover_interest = FXRolloverInterestModule(config=config)


In [ ]:
## 添加成交模型


In [ ]:
在这个回测中，我们将使用一个简单的概率成交模型。


In [ ]:
fill_model = FillModel(
    prob_fill_on_limit=0.2,
    prob_fill_on_stop=0.95,
    prob_slippage=0.5,
    random_seed=42,
)


In [ ]:
## 添加交易场所


In [ ]:
对于这个回测，我们只需要一个交易场所，它将是一个仿真的外汇ECN。


In [ ]:
SIM = Venue("SIM")
engine.add_venue(
    venue=SIM,
    oms_type=OmsType.HEDGING,  # 交易场所将生成头寸ID
    account_type=AccountType.MARGIN,
    base_currency=None,  # 多币种账户
    starting_balances=[Money(1_000_000, USD), Money(10_000_000, JPY)],
    fill_model=fill_model,
    modules=[fx_rollover_interest],
)


In [ ]:
## 添加工具和数据


In [ ]:
现在我们可以添加工具和数据。对于这个回测，我们将使用`QuoteTickDataWrangler`将买卖价K线数据预处理为报价。


In [ ]:
接下来，我们将配置并初始化一个简单的`EMACross`策略用于回测。


In [ ]:
# 配置策略
config = EMACrossConfig(
    instrument_id=USDJPY_SIM.id,
    bar_type=BarType.from_str("USD/JPY.SIM-5-MINUTE-BID-INTERNAL"),
    fast_ema_period=10,
    slow_ema_period=20,
    trade_size=Decimal(1_000_000),
)

# 实例化并添加策略
strategy = EMACross(config=config)
engine.add_strategy(strategy=strategy)


In [ ]:
## 运行回测

我们现在拥有运行回测所需的一切。一旦引擎完成处理所有数据，将记录一份后分析报告。


In [ ]:
engine.run()


In [ ]:
## 生成报告

此外，我们可以生成各种报告来进一步分析回测结果。


In [ ]:
engine.trader.generate_account_report(SIM)


In [ ]:
engine.trader.generate_order_fills_report()


In [ ]:
engine.trader.generate_positions_report()


In [ ]:
engine.trader.generate_positions_report()
